In [1]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.nn.functional as F
import numpy as np
import pandas as pd
import yfinance as yf
import filecmp
import matplotlib.pyplot as plt
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.autograd.set_detect_anomaly(True)
from sklearn import preprocessing as pre

In [2]:
start_date="2018-01-01"
end_date="2023-12-31"
tickers=["AAPL","^GSPC","MSFT"]
file='latest.csv'


In [3]:
data=pd.DataFrame()
for i in  range(len(tickers)):
    data1=yf.download(tickers[i],start=start_date,end=end_date)
    data1.rename(columns={'Open':f'{tickers[i]}_Open'},inplace=True)
    data1.rename(columns={'High':f'{tickers[i]}_High'},inplace=True)
    data1.rename(columns={'Low':f'{tickers[i]}_Low'},inplace=True)
    data1.rename(columns={'Close':f'{tickers[i]}_Close'},inplace=True)
    data1.rename(columns={'Adj Close':f'{tickers[i]}_Adj Close'},inplace=True)
    data1.rename(columns={'Volume':f'{tickers[i]}_Volume'},inplace=True)
    if data.empty:
        data=data1
    else:
        data=pd.merge(data,data1,on='Date',how='right')



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [4]:
'''with open(file, 'r') as file1:
    data_check = pd.read_csv(file1)
    
    data_check=pd.to_datetime(data_check['Date'])
if not data_check==data:
    data.to_csv(file)'''

"with open(file, 'r') as file1:\n    data_check = pd.read_csv(file1)\n    \n    data_check=pd.to_datetime(data_check['Date'])\nif not data_check==data:\n    data.to_csv(file)"

In [5]:
#data=pd.merge(data1,data2,on='Date',how='right')
#data=pd.merge(data,data3,on='Date',how='right')
data.to_csv('AAPL,MSFT,SPX.csv')
#print(data.dtypes)
data_corection=torch.as_tensor(data.values,dtype=torch.float32)
data_corection=data_corection.to(device)
data_corection=F.normalize(data_corection,p=2.0)
data.to_
data_row=data_corection.t()
data_row=data_row.to(device)
data_corection=(data_corection-torch.min(data_corection))/(torch.max(data_corection)-torch.min(data_corection))
data_corection

tensor([[7.8051e-09, 7.9625e-09, 7.7389e-09,  ..., 2.0577e-08, 1.8850e-08,
         6.6150e-03],
        [7.4532e-09, 7.5956e-09, 7.4130e-09,  ..., 1.9641e-08, 1.7978e-08,
         7.3497e-03],
        [6.9526e-09, 7.0155e-09, 6.9215e-09,  ..., 1.8843e-08, 1.7235e-08,
         5.9248e-03],
        ...,
        [6.5317e-08, 6.5684e-08, 6.4807e-08,  ..., 1.3137e-07, 1.3112e-07,
         5.4225e-03],
        [6.7220e-08, 6.7412e-08, 6.6860e-08,  ..., 1.3433e-07, 1.3408e-07,
         5.3083e-03],
        [5.7312e-08, 5.7472e-08, 5.6618e-08,  ..., 1.1557e-07, 1.1535e-07,
         5.9889e-03]], device='cuda:0')

In [12]:
class AI(nn.Module):
    def __init__(self,nUnits,nLayers):
        super(AI,self).__init__()
        
        # create dictionary to store the layers
        self.layers = nn.ModuleDict()
        self.nLayers = nLayers

        ### input layer
        self.layers['input'] = nn.Linear(18,1509)
        self.layers['hidden0']=nn.Linear(1509,nUnits)
        
        ### hidden layers
        #for i in range(nLayers):
            #self.layers[f'hidden{i+1}'] = nn.Linear(nUnits,nUnits)
        ### output layer
        self.layers['output'] = nn.Linear(nUnits,1)
    def forward(self,x):
        x=F.relu(self.layers['input'](x))
        for i in range(self.nLayers):
            x=F.relu(self.layers[f'hidden{i}'](x))
        x=self.layers['output'](x)
        return x
        


In [13]:
nUnitsPerLayer=512
nLayers=100
net=AI(1509,nLayers)
net=net.to(device)


In [14]:
target=data['AAPL_High']
target=torch.tensor(target.values,dtype=torch.float32).to(device)
target.to(device)


tensor([ 43.0750,  43.6375,  43.3675,  ..., 193.5000, 194.6600, 194.4000],
       device='cuda:0')

In [15]:
def trainTheModel(theModel,numepochs):
    lossfun=nn.L1Loss().to(device)
    optimizer=torch.optim.RMSprop(theModel.parameters(),lr=.001,momentum=.9)
    losses=torch.zeros(numepochs).to(device)
    for epochi in range(numepochs):
        print(epochi)
        yHat=theModel(data_corection)
        loss=lossfun(yHat,target)
        losses[epochi]=loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(losses)
    return losses

In [16]:
numepochs=1000
losses=trainTheModel(net,numepochs)
losses=losses.to(device='cpu')
plt.plot(losses.detach(),'o',markerfacecolor='w',linewidth=.1)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

0


KeyError: 'hidden1'